In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [ ]:
SERVER0 = ''
SERVER1 = ''
SERVER2 = ''

In [2]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 5336)
address_rst = (SERVER2, 3346)

ppl = PipelineCommon([
    (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [3]:
text = (
"При этом отправители письма напрямую связывают удары американских беспилотников с "
"серией терактов в Париже, которые произошли 13 ноября 2015 года. "
"«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки "
"в Париже, зная, какие разрушительные последствия за рубежом имеет программа "
"использования БПЛА», - говорится в открытом письме экс-военных.")

In [4]:
%%time

result = ppl(text)

CPU times: user 57.1 ms, sys: 2.86 ms, total: 59.9 ms
Wall time: 4.9 s


In [5]:
result['rst']

In [6]:
print(result['rst'][0])

id: 10
text: При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года. «Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.
proba: 0.2294643521308899
relation: elaboration
nuclearity: NS
left: При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года.
right: «Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.
start: 0
end: 359


In [7]:
vars(result['rst'][0])

{'id': 10,
 'left': <isanlp.annotation_rst.DiscourseUnit at 0x7f71343938d0>,
 'right': <isanlp.annotation_rst.DiscourseUnit at 0x7f7134393e10>,
 'relation': 'elaboration',
 'nuclearity': 'NS',
 'proba': '0.2294643521308899',
 'start': 0,
 'end': 359,
 'text': 'При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года. «Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.'}

In [8]:
def extr_pairs(tree, text):
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end], text[tree.right.start:tree.right.end], tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

In [9]:
extr_pairs(result['rst'][0], result['text'])

[['При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года.',
  '«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.',
  'elaboration',
  'NS'],
 ['При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже,',
  'которые произошли 13 ноября 2015 года.',
  'elaboration',
  'NS'],
 ['«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА»,',
  '- говорится в открытом письме экс-военных.',
  'attribution',
  'NS'],
 ['«Мы не можем спокойно сидеть',
  'и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования Б